In [ ]:
CURR_DIR = "/tf/Capstone Project/Project-2"
os.chdir(CURR_DIR)

In [ ]:
import pandas as pd
import numpy as np
import os

In [ ]:
df = pd.read_csv('./Data/prods.csv')

In [ ]:
df.head()

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13


In [ ]:
df.shape

(1000, 4)

In [ ]:
def basic_cleaning(pandas_row):
    row_in_str_form = str(pandas_row)
    row_in_lower_case = row_in_str_form.lower()
    

In [ ]:
print(df.to_numpy()[200:210])

[[201 'Sloppy Joe' 38 1]
 [202 'Spinach & Feta Pierogi' 38 1]
 [203 'Rescue Remedy, Spray' 11 11]
 [204 'Free & Clear Natural Laundry Detergent For Sensitive Skin' 75 17]
 [205 'Chocolate Mint Cookie Crunch Nutrition Bar' 3 19]
 [206 'Roasted Vegetable Soufflé' 38 1]
 [207 'Minis Candy Bars' 45 19]
 [208 'Vanilla Almond Cashew Almondmilk Cashewmilk Blend' 91 16]
 [209 'Italian Pasta Salad' 1 20]
 [210 'Homemade Hot Arrabbiata Fra Diavolo Sauce' 9 9]]
